In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = "retina"

# ResNet + Semi-supervised

In [9]:
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn import metrics

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchcontrib

# 이미지 augmentation(이미지 data 증강)을 지원해주는 일종의 라이브러리
# 어떤 논문에 따르면 torchvision이나, keras, imgaug보다 빠른 속도를 지원함.
import albumentations as A
from iterstrat import ml_stratifiers

In [10]:
# Reproduction을 위한 Seed 고정

def seed_everything(seed) :
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
seed_everything(SEED)

In [11]:
SIZE = 28
MEAN = 0.143
STD = 0.254
NUM_CALSSES = 10
PIXEL_COLS = [str(i) for i in range(784)]
DEVICE = "cuda"

# Data

In [13]:
DATA_PATH = Path("../../kaggle_data/CV_train_competition")
df = pd.read_csv(DATA_PATH/"train.csv") # Path()를 사용하여 PATH/(특정파일) 로 접근 가능

In [14]:
# 랜덤하게 몇개의 Data를 뽑고 싶을 때 -> df.sample() 함수 사용
# df.sample(n=5) -> 5 개의 row를 랜덤하게 뽑아줌.
# df.sample(frac=0.7) -> 전체 row의 70퍼센트를 뽑아온다.
# df.sample(frac=1).reset_index(drop=Ture) -> 전체 데이터의 shuffling
# reset_index -> 기존의 index가 아닌 새로운 indexing을 가능하게 함.
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
y = df[["digit", "letter"]].values
y

array([[7, 'Y'],
       [9, 'C'],
       [6, 'D'],
       ...,
       [8, 'H'],
       [8, 'M'],
       [7, 'A']], dtype=object)

In [18]:
# KFold -> target의 비율이 특정으로 몰린채로 잘려서 교차검증하면 문제가 생김
# 이를 해결하기 위해 target에 속성값 개수를 동일하게 하게 가져가는 것이 필요
# 그게 stratifiedKFold 임.
kf = ml_stratifiers.MultilabelStratifiedKFold(n_splits=5)
for fold, (train_, valid_) in enumerate(kf.split(df, y=y)) :
    np.save(f"./train_fold{fold}", train_)
    np.save(f"./valid_fold{fold}", valid_)

C:\Users\marti\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=False, random_state=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [21]:
# 모든 subcalsses 들은 주어진 key로 data fetching을 지원할 때, __getitem__()를 overwrite 해야함.
# 때에 따라 __len__()을 overwrite할 수 있다. -> 이는 Sampler 구현에 의한 Dataset의 크기와 DataLoader으 기본 옵션을 반환할 것이다.


class EMNISTDataset(torch.utils.data.Dataset) :
    def __init__(self, df, list_IDs, aug=None, label=True) :
        self.list_IDs = list_IDs
        self.label = label
        
        self.images = df[PIXEL_COLS].values
        self.images = self.images.astype(np.uint8)
        self.images = self.images.reshape(-1, SIZE, SIZE, 1)
        
        if label :
            self.digits = df.digit.values
            
        if augs is None : # 별도의 augmentation 기법 적용이 없다면
            # augmentatoin pipline을 정의하기 위해 Compose instance 생성하기
            # Compose class의 argument로 적용하고자하는 augmentations 들을 list로 넘겨줘야함.
            # Normalize -> 정규화된 augmentation 적용 인듯...?
            self.augs = A.Compose(
            [A.Normalize(MEAN, STD, max_pixel_value=255.0, always_apply=True,),]
            )
            
        else : self.augs = augs
    
    def __len__(self) :
        return len(self.list_IDs)
    
    def __getitem__(self, item) :
        # getimage
        index = self.list_IDs[item]
        image = self.images[index]
        
        # Augment image
        # compose로 정의한 augmentation을 aug(image=imgae)와 같이 넘겨서 augmentation 진행
        # augmented 한결과에서 "image" column의 결과를 가져옴
        # 그것이 augmented 한 image임.
        image = self.augs(image=image)["image"]
        
        # Convert to PyTorch tensor
        image = torch.tensor(image, dtype=torch.float)
        image = image.permute(2, 0, 1) # 축을 바꿔주는 역할을 함.
        # 0->2, 1->0, 2-> 1 로 차원을 변경해줌.
        # color channel을 torch에서 먼저 처리하는게 아닐까..?
        
        # Get labels and return
        if self.label :
            digit = self.digit[index]
            digit = torch.tensor(digit, dtype=torch.long)
            return image, digit # label이 있는 train data의 경우 image와 label 모두 반환
        else :
            return image # label이 없는 test data의 경우 image data만 반환

# Model

In [24]:
def init_cnn(m) :
    # getattr ; object에 존재하는 속성의 값을 가져옴.
    # getattr(object, 속성의 이름, 그 속성의 기본값)
    if getattr(m, "bias", None) is not None :
        nn.init.constant_(m.bias, 0) # Tensor의 값을 특정 상수로 초기화함.
    if isinstance(m, (nn.Conv2d, nn.Linear)) : # 어떤 object가 특정 형식인지 T/F 판단
        nn.init.kaiming_normal_(m.weight) # weight 초기화 방식 중의 하나
    for l in m.children() :
        init.cnn(l) #해당 module의 children들을 모두 cnn 초기화함....?

# ResNet

In [1]:
def conv3x3(in_planes, out_planes, stride=1) :
    return nn.Conv2d(
        in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False
    )

class BasicBlock(nn.Module) :
    expansion = 1
    
    def __init__(self, in_planes, planes, stride=1) :
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_planes,
                    self.expansion * planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(self.expansion * planes),
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class PreActBlock(nn.Module):
    """Pre-activation version of the BasicBlock"""

    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(PreActBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)

        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_planes,
                    self.expansion * planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                )
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        x = F.relu(self.bn1(x))
        shortcut = self.shortcut(x)
        x = self.conv1(x)
        out = self.conv2(F.relu(self.bn2(x)))
        out += shortcut
        return out


class Bottleneck(nn.Module):
    """Pre-activation version of the original Bottleneck module."""

    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(
            planes, self.expansion * planes, kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_planes,
                    self.expansion * planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(self.expansion * planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

SyntaxError: unexpected EOF while parsing (<ipython-input-1-ca88d2b24fc9>, line 6)

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(1, 64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(1024 * block.expansion, num_classes)
        init_cnn(self)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        avg_feats = F.adaptive_avg_pool2d(x, output_size=1)
        max_feats = F.adaptive_max_pool2d(x, output_size=1)
        x = torch.cat([avg_feats, max_feats], dim=1)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x


def ResNet18():
    return ResNet(PreActBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])